Title: DSC350 Week 6 Exercises  
Author: Armando Fandango  
Date: 10 November 2024  
Modified By: Caleb Trimble  
Description: This program executes various functions relevent to SQLite.  
Codes in this program have been adapted from Python Data Analysis-Second Edition (Fandango A., 2017). 

In [4]:
import sqlite3

# Creates the sensors table
with sqlite3.connect(":memory:") as con:
    c = con.cursor()
    c.execute('''CREATE TABLE sensors (date text, city text, code text, sensor_id real, temperature real)''')

    for table in c.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
        print("Table", table[0])

    c.execute("INSERT INTO sensors VALUES ('2016-11-05','Utrecht','Red',42,15.14)")
    c.execute("SELECT * FROM sensors")
    print(c.fetchone())
    con.execute("DROP TABLE sensors")

    print("# of tables", c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type = 'table'").fetchone()[0])

    c.close()

Table sensors
('2016-11-05', 'Utrecht', 'Red', 42.0, 15.14)
# of tables 0


In [5]:
import pandas as pd
import sqlite3

# Creates a list of tuples and DataFrame
data = [
    (1720, 'Moderate'),
    (1730, 'Low'),
    (1740, 'High'),
    (1750, 'Very High'),
    (1760, 'Low')
]
df = pd.DataFrame(data, columns=['Year', 'SunActivity'])
print(df)

# Connects to SQLite and create the table
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE solar_activity (
    Year,
    SunActivity
)
''')

# Inserts records and show row count
for row in data:
    cursor.execute('INSERT INTO solar_activity VALUES (?, ?)', row)
conn.commit()
cursor.execute('SELECT COUNT(*) FROM solar_activity')
row_count = cursor.fetchone()[0]
print(f"Row count: {row_count}")

# Adds a mapping for SunActivity values
activity_mapping = {'Low': 5, 'Moderate': 15, 'High': 25, 'Very High': 30}
df['ActivityValue'] = df['SunActivity'].map(activity_mapping)

# Inserts the DataFrame with ActivityValue
for index, row in df.iterrows():
    cursor.execute('INSERT INTO solar_activity (Year, SunActivity) VALUES (?, ?)', (row['Year'], row['ActivityValue']))
conn.commit()

# Deletes records where sun activity is greater than 20
cursor.execute('DELETE FROM solar_activity WHERE SunActivity > 20')
conn.commit()

# Selects data where the year is less than 1732
cursor.execute('SELECT * FROM solar_activity WHERE Year < 1732')
results = cursor.fetchall()
for row in results:
    print(row)

# Closes the connection
conn.close()


   Year SunActivity
0  1720    Moderate
1  1730         Low
2  1740        High
3  1750   Very High
4  1760         Low
Row count: 5
(1720, 15)
(1730, 5)


In [8]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker

# Defines the superclass
Base = declarative_base()

# Creates the DBSession
engine = create_engine('sqlite:///example.db')
Session = sessionmaker(bind=engine)
session = Session()

# Creates models and commits the Session
class Station(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    name = Column(String)

class Sensor(Base):
    __tablename__ = 'sensors'
    id = Column(Integer, primary_key=True)
    station_id = Column(Integer)
    sensor_type = Column(String)
    value = Column(Float)

# Creates the tables
Base.metadata.create_all(engine)

# Creates two station records
station1 = Station(name="Station A")
station2 = Station(name="Station B")

# Adds and commits the stations
session.add(station1)
session.add(station2)
session.commit()

# Inserts a Sensor Record
sensor1 = Sensor(station_id=station1.id, sensor_type="Temperature", value=23.5)

# Adds and s the sensor record
session.add(sensor1)
session.commit()


In [7]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd

# Creates an Engine from a URI
engine = create_engine('sqlite:///example.db')

# Creates the Station and Sensor Tables
Base = declarative_base()

class Station(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    name = Column(String)

class Sensor(Base):
    __tablename__ = 'sensors'
    id = Column(Integer, primary_key=True)
    station_id = Column(Integer, ForeignKey('stations.id'))
    sensor_type = Column(String)
    value = Column(Float)

Base.metadata.create_all(engine)

# Creates a DBSession Object
Session = sessionmaker(bind=engine)
session = Session()

# Selects the First Row in the Station Table
first_station = session.query(Station).first()
print(first_station)

# Selects All the Stations
all_stations = session.query(Station).all()
for station in all_stations:
    print(station)

# Selects All the Sensors
all_sensors = session.query(Sensor).all()
for sensor in all_sensors:
    print(sensor)

# Selects the First Sensor, Which Belongs to the First Station
first_sensor_of_first_station = session.query(Sensor).filter(Sensor.station_id == first_station.id).first()
print(first_sensor_of_first_station)

# Queries the Station Table with pandas read_sql()
df_stations = pd.read_sql('SELECT * FROM stations', engine)
print(df_stations)


   id       name
0   1  Station A
1   2  Station B


C:\Users\caleb\AppData\Local\Temp\ipykernel_3200\1042057212.py:10: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
